In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
from azureml.core.webservice import AksWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
import json

In [ ]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [ ]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Score Script

In [ ]:
%%writefile ./src/score.py

import numpy as np
import pandas as pd
import json
import os
import joblib

from azureml.core.model import Model

from pipe import create_pipeline


def init():
    global model
    model_path = Model.get_model_path('SKLearnClassifier')
    model = joblib.load(model_path)
    
    global regressor
    regressor_path = Model.get_model_path('SKLearnRegressor')
    regressor = joblib.load(regressor_path)
    
    global pipelines
    pipeline_path = Model.get_model_path('SKLearnPreprocessPipeline')
    pipelines = joblib.load(pipeline_path)
    
def run(raw_data):
    
    # get input data
    data = json.loads(raw_data)
    
    # transform with pipeline
    X = pipelines['feature_pipe'].transform(pd.DataFrame(data))
    
    # make prediction
    y = model.predict(X)
    
    # predict duration
    y_dur = regressor.predict(X)
    
    response = [
        {
            'Products':
            [ 
                pipelines['target_pipe'].transformer_list[1][1].named_steps['target_encode'].col_cats[0][i] 
                for i in range(y.shape[1]) if y[j,i] == 1 
            ],
            'Duration':
                 y_dur[j,0]
        }        
            for j in range(y.shape[0])
    ]

    return response

## Deploy Webservice

In [ ]:
inference_config = InferenceConfig(entry_script="score.py", environment=ws.environments[cfg['env_name']], source_directory='./src')

aksconfig = AksWebservice.deploy_configuration()

service = Model.deploy(workspace=ws, 
                           name=cfg['webservice_name'],
                           models= [ws.models[cfg['PreprocessPipeline']], ws.models[cfg['TrainedClassifier']], ws.models[cfg['TrainedRegressor']]], 
                           inference_config=inference_config, 
                           deployment_config=aksconfig,
                           deployment_target=ws.compute_targets[cfg['compute_target']])

service.wait_for_deployment()